In [ ]:
import os

folder_path = '../src'
os.chdir(folder_path)
from dataset_utils.download_data import download_cloudsen12plus
from utils.data_loader import create_dataloaders
import tacoreader
import pandas as pd

In [ ]:
# Baixar os dados (caso ainda não estejam baixados)
parts = download_cloudsen12plus(local_dir="../data/dados", type = "L1C")
print("Arquivos baixados:", parts)

Arquivos baixados: ['..\\data\\dados\\cloudsen12-l1c.0000.part.taco', '..\\data\\dados\\cloudsen12-l1c.0004.part.taco']


In [3]:
# Carregar o dataset
ds = tacoreader.load(parts)
print("Número total de amostras:", len(ds))

Número total de amostras: 16135


In [4]:
print(ds["tortilla:data_split"].value_counts())

tortilla:data_split
train         13248
test           1715
validation     1172
Name: count, dtype: int64


- A contagem global (47360 train / 1715 test / 1172 validation) é o total de todas as amostras do CloudSEN12+

In [5]:
ds = tacoreader.load(parts)
df = pd.DataFrame(ds)
print(df.columns)  
print(df.head())  

Index(['internal:subfile', 'tortilla:id', 'tortilla:file_format',
       'tortilla:data_split', 'tortilla:offset', 'tortilla:length', 'stac:crs',
       'stac:geotransform', 'stac:raster_shape', 'stac:time_start',
       'stac:time_end', 'stac:centroid', 'rai:ele', 'rai:cisi', 'rai:gdp',
       'rai:hdi', 'rai:gmi', 'rai:pop', 'rai:admin0', 'rai:admin1',
       'rai:admin2', 'roi_id', 'old_roi_id', 'equi_id', 'equi_zone',
       'label_type', 's2_id', 'real_proj_shape', 's2_mean_solar_azimuth_angle',
       's2_mean_solar_zenith_angle', 'thick_percentage', 'thin_percentage',
       'cloud_shadow_percentage', 'clear_percentage'],
      dtype='object')
                                       internal:subfile  \
1498  /vsisubfile/10460113754_1437347,..\data\dados\...   
1499  /vsisubfile/10461551101_1288233,..\data\dados\...   
1500  /vsisubfile/10462839334_1437179,..\data\dados\...   
1501  /vsisubfile/10464276513_1504849,..\data\dados\...   
1502  /vsisubfile/10465781362_1501969,..\data\

**Treinar primeiro em p509** (que possui mais amostras e, portanto, fornece uma diversidade ampla para a rede aprender as classes de nuvem).  
**Depois refinar (fine-tuning) com p2000**, pois os patches maiores ajudam a capturar melhor as correlações espaciais entre nuvens e sombras, o que é benéfico especialmente para a detecção de sombras.

Em resumo:
  
**Treino principal em p509**: aproveita-se a grande quantidade de patches com rótulos “high” (além de “scribble” e “nolabel”, se desejar) para uma base sólida.  
**Fine-tuning em p2000**: complementa o aprendizado, fornecendo amostras bem maiores, nas quais o modelo pode aprender melhor a relação geométrica entre nuvens e suas sombras.

Em termos de implementação, você pode:
- Primeiramente carregar apenas p509 (“real_proj_shape=509”) nos seus DataLoaders, treinar até convergência.  
- Em seguida, iniciar um treinamento (ou fine-tuning) carregando apenas p2000 (“real_proj_shape=2000”). Nesse ponto, ou você retoma o estado do modelo anterior (carregando o checkpoint da rede treinada em p509) e faz alguns epochs extras, ou congela camadas, ou usa outra estratégia de transferência, conforme a arquitetura e práticas de fine-tuning que preferir.

No artigo, os **três tipos de label_type** aparecem descritos assim:

**high**  
   - Patches com anotações densas (cada pixel é classificado em clear, thick cloud, thin cloud ou cloud shadow).  
   - Divididos em train, val, e test.  
   - Ideais para **treinamento supervisionado** tradicional, pois cada pixel tem um rótulo confiável.  

**scribble**  
   - Patches onde apenas um pequeno conjunto de pixels (menos de 5%) foi anotado manualmente, em formato de “rabiscos” (daí o nome scribble).  
   - Também divididos em train, val e test.  
   - Úteis para **validação** e para experimentos de semi-supervisão, pois oferecem alguma informação de rótulo em poucos pixels, ajudando a corrigir erros em regiões críticas (bordas de nuvem, por exemplo).

**nolabel**  
   - Patches **sem** anotações humanas.  
   - Disponíveis apenas na pasta *train*.  
   - O artigo menciona que cada um desses patches recebe uma máscara de nuvens inferida automaticamente pelo modelo UnetMobV2, podendo servir como base para pré-treinamento (pseudo-rótulos) ou para estratégias de aprendizado semi-supervisionado.  

Em suma, **“high”** é o principal conjunto supervisionado, **“scribble”** serve como conjunto adicional de validação/treinamento parcial, e **“nolabel”** oferece um grande volume de dados sem anotação manual, mas com máscaras geradas automaticamente para quem quiser explorar técnicas de pseudo-rótulo, auto-treinamento ou aprendizado semi-supervisionado.

In [6]:
train_loader, val_loader, test_loader = create_dataloaders(
    parts,
    real_proj_shape=509,  # ou 2000 
    label_type="high",    # "scribble" ou "nolabel"
    batch_size=8,
    num_workers=2
)
# Verificar quantos batches e a forma dos tensores
if train_loader is not None:
    for idx, (imgs, masks) in enumerate(train_loader):
        print(f"Lote {idx}: imagens={imgs.shape}, máscaras={masks.shape}")
        break

Train samples: 8490
Val samples:   535
Test samples:  975
Lote 0: imagens=torch.Size([8, 13, 512, 512]), máscaras=torch.Size([8, 1, 512, 512])


- subconjunto (8490 train / 535 val / 975 test) surge depois que aplica os filtros para:

    - real_proj_shape = 509 (ou seja, p509, descartando os patches de 2000×2000)
    - label_type = "high" (descartando scribble e nolabel)
    - tortilla:data_split in {"train", "validation", "test"}

Geralmente, a ordem (para o Sentinel-2 L1C) é a seguinte:

B1: Coastal Aerosol

B2: Blue

B3: Green

B4: Red

B5: Vegetation Red Edge 1

B6: Vegetation Red Edge 2

B7: Vegetation Red Edge 3

B8: NIR

B8A: Narrow NIR

B9: Water Vapor

B10: Cirrus

B11: SWIR 1

B12: SWIR 2